# List intersection algorithms

We experiment two list intersection algorithms here. 


In [67]:
#DEBUG = True
DEBUG = False

def p(msg):
    if DEBUG:
        print(".. {}".format(msg)) 

## The InvertedList class

We implement this class which encapsulates all the important *operations* one can perform on an inverted list. 

In [68]:
from pprint import pprint
from bisect import *

class InvertedList:
    def __init__(self, l):
        self.data = l[:] # make a copy
        self.cur = 0

    def get_list(self):
        return self.data
 
    def eol(self):
        # we use cur == len(list) to indicate EOL
        return False if self.cur < len(self.data) else True
    
    def next(self, val = 1):
        # does not allow cur to be out-of-range, but use len(list) to indicate EOL
        self.cur = min(self.cur + val, len(self.data)) 
            
    def elem(self):
        if self.eol():
            return None
        else: 
            return self.data[self.cur]

    def peek(self, pos):
        if pos < len(self.data):
            return self.data[pos]
        else:
            return None
        
    def reset(self):
        self.cur = 0
    
    def dump(self):
        print("* {} : {} (cur = {})".format(self.data[:self.cur], self.data[self.cur:], self.cur))
        
    def skip_to(self, val):
        # move the cursor to the first element no smaller than `val`
        while not self.eol() and self.elem() < val:
            self.next()               
            
    def gallop_to(self, val):
        # move the cursor to the first element no smaller than `val`

        ## stage 1: 
        interval = 1
        pos = self.cur
        last_pos = 0
        cur_val = self.peek(pos)
        if cur_val != None and cur_val < val:
            while True:
                interval *= 2
                last_pos = pos
                pos = pos + interval
                cur_val = self.peek(pos)
                p("moved to pos = {}, elem = {}".format(pos, self.peek(pos)))
                if cur_val == None or cur_val >= val:
                    break
        # out of the 1st loop
        # possibility 1: end of the list
        abort = False
        if pos > len(self.data):
            # test the last elem
            if self.data[-1] < val:
                # val is larger than any of the element => just exhaust the list
                self.cur = len(self.data)
                abort = True
            else:
                pos = len(self.data) - 1 # just set to the last element
        ## debugging
        p("Stage 1 complete: interval = [{}, {}], elems = [{}, {}]".format(
            last_pos, pos, self.peek(last_pos), self.peek(pos)))
        
        if abort == False and pos - last_pos > 0:
            ## stage 2
            sub_list = self.data[(last_pos + 1) : pos + 1] # last_pos + 1 as this pos has been searched
            sub_pos = bisect_left(sub_list, val) # won't return None
            self.cur = (last_pos + 1) + sub_pos #! move the cursor
            ## debugging
            p("Stage 2 complete: cur = {}, sub_pos = {}, sub_list = {}".format(self.cur, sub_pos, sub_list))

                        





### Test the class

In [69]:
a = InvertedList([2, 4, 6, 8, 10, 12, 14, 16, 18])
b = InvertedList([1, 2, 4, 8, 16, 32])

while not a.eol():
    e = a.elem()
    a.next()
    print(e)

a.dump()

2
4
6
8
10
12
14
16
18
* [2, 4, 6, 8, 10, 12, 14, 16, 18] : [] (cur = 9)


Test `gallop_to()`

In [70]:
def test_gallop(l, val):
    print("=> gallop_to({})".format(val))
    l.reset()
    l.gallop_to(val)
    l.dump()

In [71]:
#test_a = [-1].append( [val + 1 for val in a.get_list()] )
test_a = [val - 1 for val in a.get_list()]
test_a.append(a.get_list()[-1] + 100)

DEBUG = False
a.dump()
for t_a in test_a:
    test_gallop(a, t_a)

* [2, 4, 6, 8, 10, 12, 14, 16, 18] : [] (cur = 9)
=> gallop_to(1)
* [] : [2, 4, 6, 8, 10, 12, 14, 16, 18] (cur = 0)
=> gallop_to(3)
* [2] : [4, 6, 8, 10, 12, 14, 16, 18] (cur = 1)
=> gallop_to(5)
* [2, 4] : [6, 8, 10, 12, 14, 16, 18] (cur = 2)
=> gallop_to(7)
* [2, 4, 6] : [8, 10, 12, 14, 16, 18] (cur = 3)
=> gallop_to(9)
* [2, 4, 6, 8] : [10, 12, 14, 16, 18] (cur = 4)
=> gallop_to(11)
* [2, 4, 6, 8, 10] : [12, 14, 16, 18] (cur = 5)
=> gallop_to(13)
* [2, 4, 6, 8, 10, 12] : [14, 16, 18] (cur = 6)
=> gallop_to(15)
* [2, 4, 6, 8, 10, 12, 14] : [16, 18] (cur = 7)
=> gallop_to(17)
* [2, 4, 6, 8, 10, 12, 14, 16] : [18] (cur = 8)
=> gallop_to(118)
* [2, 4, 6, 8, 10, 12, 14, 16, 18] : [] (cur = 9)


In [72]:
DEBUG = True
test_gallop(a, test_a[-2])

=> gallop_to(17)
.. moved to pos = 2, elem = 6
.. moved to pos = 6, elem = 14
.. moved to pos = 14, elem = None
.. Stage 1 complete: interval = [6, 8], elems = [14, 18]
.. Stage 2 complete: cur = 8, sub_pos = 1, sub_list = [16, 18]
* [2, 4, 6, 8, 10, 12, 14, 16] : [18] (cur = 8)


As you can see above, the galloping search algorithm quickly skip over the list to reposition the cursor. 

## The Naive Intersection Algorithm

In [73]:
def intersection_naive(a, b):
    # just in case these lists have been traversed.
    a.reset()
    b.reset()
    
    ret = []
    while not a.eol() and not b.eol():
        if a.elem() < b.elem():
            p("move a")
            a.next()
        elif a.elem() > b.elem():
            p("move b")
            b.next()
        else:
            p("found one")
            ret.append(a.elem())
            a.next()
            b.next()
    return ret
        

Test

In [74]:
intersection_naive(a, b)

.. move b
.. found one
.. found one
.. move a
.. found one
.. move a
.. move a
.. move a
.. found one
.. move a


[2, 4, 8, 16]

## Intersection Algorithm using `gallop_to()`

In [75]:
def intersection_galloping(a, b):
    # just in case these lists have been traversed.
    a.reset()
    b.reset()
    
    ret = []
    while not a.eol() and not b.eol():
        if a.elem() == b.elem():
            p("found one")
            ret.append(a.elem())
            a.next()
        else:
            if a.elem() < b.elem():
                a.gallop_to(b.elem())
            else:
                b.gallop_to(a.elem())
    # end_while
    return ret
        

In [76]:
intersection_galloping(a, b)

.. moved to pos = 2, elem = 4
.. Stage 1 complete: interval = [0, 2], elems = [1, 4]
.. Stage 2 complete: cur = 1, sub_pos = 0, sub_list = [2, 4]
.. found one
.. moved to pos = 3, elem = 8
.. Stage 1 complete: interval = [1, 3], elems = [2, 8]
.. Stage 2 complete: cur = 2, sub_pos = 0, sub_list = [4, 8]
.. found one
.. moved to pos = 4, elem = 16
.. Stage 1 complete: interval = [2, 4], elems = [4, 16]
.. Stage 2 complete: cur = 3, sub_pos = 0, sub_list = [8, 16]
.. moved to pos = 4, elem = 10
.. Stage 1 complete: interval = [2, 4], elems = [6, 10]
.. Stage 2 complete: cur = 3, sub_pos = 0, sub_list = [8, 10]
.. found one
.. moved to pos = 5, elem = 32
.. Stage 1 complete: interval = [3, 5], elems = [8, 32]
.. Stage 2 complete: cur = 4, sub_pos = 0, sub_list = [16, 32]
.. moved to pos = 6, elem = 14
.. moved to pos = 10, elem = None
.. Stage 1 complete: interval = [6, 8], elems = [14, 18]
.. Stage 2 complete: cur = 7, sub_pos = 0, sub_list = [16, 18]
.. found one
.. moved to pos = 6, el

[2, 4, 8, 16]